In [ ]:
from bg_atlasapi import BrainGlobeAtlas
from pprint import pprint
# this downloads the atlas and meshes,we could also do it straight from the allen but this is easier.
bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)  

In [81]:
from tifffile import imread
import meshio
import json
from pathlib import Path
import numpy as np
import pyvista as pv # for plotting 3d
pv.set_jupyter_backend('trame')

# load the atlas metadata
atlaslocation =  Path('~').expanduser()/'.brainglobe'/'allen_mouse_25um_v1.2/'
with open(atlaslocation/'structures.json','r') as fd:
    structures = json.load(fd)
with open(atlaslocation/'metadata.json','r') as fd:
    metadata = json.load(fd)

with open(atlaslocation/'structures.json','r') as fd:
    structures = json.load(fd)

# AP,DV,ML
bregma_location = np.array([216, 18,228 ])*metadata['resolution'] #don't trust too much, for the 25um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858

#bregma_location = [540, 44, 570] #don't trust too much, for the 10um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858



import pandas as pd
structures = pd.DataFrame(structures)

# load meshes
def load_structure_mesh(atlaslocation,structures,acronym):
    # meshes are in um
    id = structures[structures.acronym == acronym].id.values
    if len(id):
        id = id[0]
    else:
        return
    mesh = atlaslocation/'meshes'/f'{id}.obj'
    mesh = pv.read(mesh)
    return mesh, structures[structures.acronym == acronym].iloc[0]
root,rootstruct = load_structure_mesh(atlaslocation,structures,'root')

toplot = []
for acronym in ['CP','LP','VISp','VISa','SCs','ACA','MD']:
    toplot.append(load_structure_mesh(atlaslocation,structures,acronym))

# todo, translate and rotate the meshes so they are in stereotaxic position.

# p = pv.Plotter(window_size = [1000,1000])# for saving in the next cell with high res
p = pv.Plotter(window_size = [600,400])

axes = p.add_axes()
axes.origin = bregma_location

rotate5deg = True
if rotate5deg:
    root.rotate_z(5,point=axes.origin, inplace=True)

# There are 2 ways of doing this, transform the meshes of transform the probes, lets transform the probes,
# the other way silhoette doesnt work.
# M = np.array([[0.025, 0., 0., 0.],
#                    [0., 0.025, 0., 0.],
#                    [0., 0., 0.025, 0.],
#                    [0., 0., 0., 1.]])
p.add_mesh(root, color = rootstruct['rgb_triplet'], opacity = 0.1,silhouette=False)#,user_matrix=M)
for s in toplot:
    if rotate5deg:
        s[0].rotate_z(5,point=axes.origin, inplace=True)
    c = s[0].clip('z', invert=True)
    p.add_mesh(c,color=s[1]['rgb_triplet'],opacity = 0.7, silhouette=dict(color='#000000',line_width=1) )

# todo, make all probes start at the same depth (needs angles)
probes_position_type = [(np.array([-2394,781,-991]),np.array([-1278,2197,-192]),4,0),
                        (np.array([-1665,-646,1898]),np.array([1210,3664,-1022]),4,0),
                        (np.array([131,646,-4224]),np.array([431,4235,-2819]),4,0),
                        (np.array([131,646,-4224]),np.array([431,4235,-2819]),4,0),
                        (np.array([1288,103,-1132]),np.array([1862,3472,-1286]),4,-130),
                        (np.array([4215,-200,-2241]),np.array([2628,3681,-1501]),4,90),
                        (np.array([3489,-292,1176]),np.array([3489,2074,-1377]),4,0)]

for start,tip,nshanks,spin in probes_position_type:
    # Can only plot start and end locations
    line = pv.Line(pointa = start+bregma_location, pointb = tip+bregma_location)
    p.add_mesh(line,color='#000000',opacity = 1,line_width=3)
    for s in range(1,4):
        start,tip = make_20_shanks(start,tip,1,spin = spin)
        line = pv.Line(pointa = start+bregma_location, pointb = tip+bregma_location)
        p.add_mesh(line,color='#000000',opacity = 1,line_width=3)
p.show()


Widget(value='<iframe src="http://localhost:37097/index.html?ui=P_0x7fb1284a1050_38&reconnect=auto" class="pyv…

In [80]:
p.save_graphic('/home/joao/Desktop/JC140_probes_planned_rotated.pdf')
# start,tip = (np.array([-1665,-646,1898]),np.array([1210,3664,-1022]))





In [69]:
p.camera_position

[(6549.170102581264, -31986.954180910107, 11921.351433090116),
 (6293.4735693589, 3967.848571900686, 5688.1645),
 (-0.05838422265325471, 0.1701194078712309, 0.9836923653311117)]

In [61]:
# start,tip,nshanks = probes_position_type[0]
# x1,y1,z1 = start
# x2,y2,z2 = tip
# tt = np.rad2deg(np.arctan2(x1*y2 - y1*x2,x1*x2 + y1+y2)),np.rad2deg(np.arctan2(x1*z2 - z1*x2,x1*x2 + z1+z2))
def rotation_matrix_from_degrees(yaw_degrees, pitch_degrees, roll_degrees):
    """Return a rotation matrix to rotate a vector in 3D space. Pass the angles in degrees, not radians."""
    from math import cos, sin, radians
    alpha = radians(yaw_degrees)
    beta = radians(pitch_degrees)
    gamma = radians(roll_degrees)

    Rz = np.array([[cos(alpha), -sin(alpha), 0],
                     [sin(alpha), cos(alpha), 0],
                     [0, 0, 1]])
    Ry = np.array([[cos(beta), 0, sin(beta)],
                   [0, 1, 0],
                   [-sin(beta), 0, cos(beta)]])
    Rx = np.array([[1,0,0],
                   [0, cos(gamma), -sin(gamma)],
                   [0, sin(gamma), cos(gamma)]])

    return Rz @ Ry @ Rx

def make_20_shanks(start,tip,shank,spin = 0):
    # apply shank offsets and rotate coordinate frame
    NP2_SHANK_DISTANCE_MM = np.array([0,0,250]) 
    x1,y1,z1 = start
    x2,y2,z2 = tip
    additional_shank_positions = []
    additional_shank_angles = []
    elevation,azimuth = np.rad2deg(np.arctan2(x1*y2 - y1*x2,x1*x2 + y1+y2)),np.rad2deg(np.arctan2(x1*z2 - z1*x2,x1*x2 + z1+z2))
    
    rotation_matrix = rotation_matrix_from_degrees(elevation,azimuth,spin)
    shank1loc = np.dot(rotation_matrix, NP2_SHANK_DISTANCE_MM * shank) + np.array([y1,z1,x1])
    shank2loc = np.dot(rotation_matrix, NP2_SHANK_DISTANCE_MM * shank) + np.array([y2,z2,x2])
    y1,z1,x1 = shank1loc
    y2,z2,x2 = shank2loc
    return np.array([x1,y1,z1]),np.array([x2,y2,z2])
# make_20_shanks(start,tip,1)

In [22]:
start = np.array([-2394.40602769,780.70820982,-991.])
tip = np.array([-1278.20301385,  2196.85410491 ,-192.])
line = pv.Line(pointa = start+bregma_location, pointb = tip+bregma_location)
p.add_mesh(line,color='#000000',opacity = 0.9,line_width=3)

Actor (0x7fb481ea43a0)
  Center:                     (3563.6954345703125, 1938.7811279296875, 5108.5)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    3.006E+03, 4.122E+03
  Y Bounds                    1.231E+03, 2.647E+03
  Z Bounds                    4.709E+03, 5.508E+03
  User matrix:                Set
  Has mapper:                 True

Property (0x7fb47bbf2a40)
  Ambient:                     0.0
  Ambient color:               Color(name='black', hex='#000000ff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(name='black', hex='#000000ff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='black', hex='#000000ff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  Edge opacity:     